[![View in Deepnote](https://deepnote.com/static/buttons/view-in-deepnote-white.svg)](https://deepnote.com/viewer/github/katetruman/MultiAgentEC/blob/master/Visualisation/VisualisationIndividualExpectations.ipynb)

# Individual expectations and Visualisation example

This notebook explores expectations and a visualisation example related to our organ transplant scenario. If you have not yet done so, please refer to **AgentsAndEvents.ipynb** for important events and fluents used in this scenario, and **Expectations.ipynb** for a discussion of expectations in this context. Both of these notebooks are located in the parent repository, **MultipleAgentsEC**.

### Set up

We need to set up our environment by loading in our event and fluent declarations from **dec:notation.pl** and **AE.pl**. The latter contains the code from **AgentsAndEvents.ipynb** the last time that notebook was run.

```diff
- IMPORTANT NOTE: you should run AgentsAndEvents.ipynb prior to running this notebook to ensure that AE.pl contains up to date predicates! You must run the notebook, not just update and save it.
```

In [1]:
?- cd('~/work'), ['dec:notation'].
?- initialiseDEC.
?- retractall(happensAtNarrative(_,_)).
?- ['AE'].

true.
false.
true.
true.

**Individual hospital agents may have expectations specific to a given scenario that do not fit into one of our other expectation categories. Examples of some organ transplant specific expectations are shown below. We will then visualise our scenario using Graphviz.**

Otago hospital expects that once a request to add a patient to the organ waiting list has been accepted, they will not have to wait more than 25 time periods for a transplant:

In [2]:
% File: maxWaitingTime.pl
initially("Otago":exp_rule(happ(receive(monSys, waitAccept(ID))), within(and([happ(receive(Location, transplantOutcome(MatchID, _))), 
donorFound(MatchID, ID, _, Location, _, _)]), 25), dependent, 
"Individual":"Patients will not have to wait more than 25 time periods for a transplant. ID":ID)).

Wellington hospital expects that waiting list add requests will not involve more than two organs per patient:

In [3]:
% File: maximumOrgans.pl
initially("Wellington":exp_rule(happ(send(monSys, waitAddReq(ID, Organs, _))), and([condition(length(Organs, Len)), condition(Len =< 2)]), dependent, 
"Individual":"Patients will not be added to the waiting list for more than 2 organs at once. ID":ID)).

Auckland hospital expects that donor Offers will not be made for patients with cancer:

In [4]:
% File: noCancerOffers.pl
initially("Auckland":exp_rule(happ(send(monSys, donorOffer(PatID, Organs, Details))), not(and([condition(member(condition:D, Details)), 
condition(member("Cancer", D))])), dependent, "Individual":"Donor offers will not be made for patients diagnosed with cancer. ID":PatID)).

Our test narrative is as follows:
- At time period 1, Otago sends a waiting list add request for patient 101 to **monSys**.
- At time period 3, Otago sends a waiting list add request for patient  to **monSys**.
- At time period 5, Wellington sends a waiting list add request for 2 organs for a patient to **monSys**.
- At time period 7, Wellington sends a waiting list add request for 3 organs for a patient to **monSys**.
- At time period 9, Auckland sends a donor offer for a patient with cancer to **monSys**.
- At time period 11, Auckland sends a donor offer for patient 202 to **monSys**.
- At time period 13, **monSys** matches the donor offer from patient 202 from Auckland to waiting patient 102 from Otago.

In [5]:
% File: narrative.pl
happensAtNarrative("Otago":send(monSys, waitAddReq(101, ["kidney", "liver"], [bloodType:"O"])),1).
happensAtNarrative("Otago":send(monSys, waitAddReq(102, ["kidney", "heart", "liver"], [bloodType:"A"])),3).
happensAtNarrative("Wellington":send(monSys, waitAddReq(103, ["heart", "pancreas"], [bloodType:"B"])),5).
happensAtNarrative("Wellington":send(monSys, waitAddReq(104, ["heart", "pancreas", "lungs"], [bloodType:"AB"])),7).
happensAtNarrative("Auckland":send(monSys, donorOffer(201, ["heart"], [bloodType:"A", condition:["HIV", "Cancer"]])), 9).
happensAtNarrative("Auckland":send(monSys, donorOffer(202, ["kidney", "liver", "pancreas"], [bloodType:"O"])),11).
happensAtNarrative(monSys:match(701, "Auckland", 202, "Otago", 102, "Wellington", ["kidney", "liver"], []),13).
happensAtNarrative("Auckland":send(monSys, acceptMatch(701)), 15).
happensAtNarrative("Wellington":send(monSys, acceptMatch(701)), 15).
happensAtNarrative("Otago":send(monSys, acceptMatch(701)), 15).
happensAtNarrative("Wellington":transplant(701, success), 20).

In [6]:
?- run(30).

true.

At time period 4, Otago is notified that their waiting list add request for patient 101 has been accepted. This creates the expectation that a donor match will be found for patient 101 within 25 time periods. Likewise, at time period 6 Otago is notified that their add request for patient 102 has been accepted, which creates another expectation.

In [7]:
?- T = 4, happensAt("Otago", Event, T).
?- T = 4, holdsAt("Otago", exp(_,_,_,Condition,_,Message), T).
?- T = 6, happensAt("Otago", Event, T).
?- T = 6, holdsAt("Otago", exp(_,_,_,Condition,_,Message), T).

T = 4, Event = receive(monSys, waitAccept(101)) .
T = 4, Condition = within(and([Functor(14504077,1,receive(_1758, transplantOutcome(_1764, _1766))), Functor(14639885,6,_1764,101,_1780,_1758,_1784,_1786)]), 25), Message = :(b'Individual', :(b'Patients will not have to wait more than 25 time periods for a transplant. ID', 101)) .
T = 6, Event = receive(monSys, waitAccept(102)) .
T = 6, Condition = within(and([Functor(14504077,1,receive(_1758, transplantOutcome(_1764, _1766))), Functor(14639885,6,_1764,101,_1780,_1758,_1784,_1786)]), 23), Message = :(b'Individual', :(b'Patients will not have to wait more than 25 time periods for a transplant. ID', 101)) ;
T = 6, Condition = within(and([Functor(14504077,1,receive(_1758, transplantOutcome(_1764, _1766))), Functor(14639885,6,_1764,102,_1780,_1758,_1784,_1786)]), 25), Message = :(b'Individual', :(b'Patients will not have to wait more than 25 time periods for a transplant. ID', 102)) .

At time period 5, Wellington sends a waiting list add request for a patient needing two organs. This fulfils the expectation that a waiting list add request will not be for more than 2 organs. At time period 7, Wellington sends a waiting list add request for a patient needing three organs, which violates the expectation.

In [8]:
?- T = 5, happensAt("Wellington", Event, T), Event = send(_, _).
?- T = 5, happensAt("Wellington", fulf(_ ,_, _, FulfCondition, _, Message), T).
?- T = 7, happensAt("Wellington", Event, T), Event = send(_,_).
?- T = 7, happensAt("Wellington", viol(_, _, _, ViolCondition, _, Message), T).

T = 5, Event = send(monSys, waitAddReq(103, [b'heart', b'pancreas'], [Functor(188685,2,bloodType,b'B')])) .
T = 5, FulfCondition = and([Functor(14479501,1,length([b'heart', b'pancreas'], 2)), Functor(14479501,1,=<(2, 2))]), Message = :(b'Individual', :(b'Patients will not be added to the waiting list for more than 2 organs at once. ID', 103)) .
T = 7, Event = send(monSys, waitAddReq(104, [b'heart', b'pancreas', b'lungs'], [Functor(188685,2,bloodType,b'AB')])) .
T = 7, ViolCondition = and([Functor(14479501,1,length([b'heart', b'pancreas', b'lungs'], 3)), Functor(14479501,1,=<(3, 2))]), Message = :(b'Individual', :(b'Patients will not be added to the waiting list for more than 2 organs at once. ID', 104)) .

When Auckland sends a donor offer for a patient with cancer at time period 9, it violates the relevant expectation. When Auckland sends a donor offer for a different patient who is not noted to have cancer at time period 11, the relevant expectation is fulfilled.

In [9]:
?- T = 9, happensAt("Auckland", Event, T), Event = send(_, _).
?- T = 9, happensAt("Auckland", viol(_, _, _, ViolCondition, _, Message), T).
?- T = 11, happensAt("Auckland", Event, T), Event = send(_, _).
?- T = 11, happensAt("Auckland", fulf(_, _, _, FulfCondition, _, Message), T).

T = 9, Event = send(monSys, donorOffer(201, [b'heart'], [Functor(188685,2,bloodType,b'A'), Functor(188685,2,condition,[b'HIV', b'Cancer'])])) .
T = 9, ViolCondition = not(and([Functor(14479501,1,member(:(condition, [b'HIV', b'Cancer']), [Functor(188685,2,bloodType,b'A'), Functor(188685,2,condition,[b'HIV', b'Cancer'])])), Functor(14479501,1,member(b'Cancer', [b'HIV', b'Cancer']))])), Message = :(b'Individual', :(b'Donor offers will not be made for patients diagnosed with cancer. ID', 201)) .
T = 11, Event = send(monSys, donorOffer(202, [b'kidney', b'liver', b'pancreas'], [Functor(188685,2,bloodType,b'O')])) .
T = 11, FulfCondition = not(and([Functor(14479501,1,member(:(condition, [b'Cancer']), [Functor(188685,2,bloodType,b'O')])), Functor(14479501,1,member(b'Cancer', [b'Cancer']))])), Message = :(b'Individual', :(b'Donor offers will not be made for patients diagnosed with cancer. ID', 202)) .

At time period 13, **monSys** matches the donor offer for patient 202 from Auckland to the waiting patient 102 from Otago, and specifies the transplant location as Wellington. This causes messages about the match to be sent to each of the three hospitals, which all send acceptance messages at time period 15. The transplant successfully takes place at time period 20, and at time period 21 Wellington sends a message to **monSys** and Otago and Auckland hospitals, notifying them of the transplant.

In [10]:
?- T = 20, happensAt("Wellington", Event, T).
?- T = 21, happensAt("Wellington", Event, T).

T = 20, Event = transplant(701, success) .
T = 21, Event = send(b'Auckland', transplantOutcome(701, success)) ;
T = 21, Event = send(b'Otago', transplantOutcome(701, success)) ;
T = 21, Event = send(monSys, transplantOutcome(701, success)) .

At time period 22, Otago receives the `transplantOutcome` notification, which fulfils its expectation that patient 102 will not have to wait more than 25 periods for an organ donation.

In [11]:
?- T = 22, happensAt("Otago", Event, T).

T = 22, Event = fulf(happ(receive(monSys, waitAccept(102))), within(and([Functor(14504077,1,receive(b'Wellington', transplantOutcome(701, success))), Functor(14639885,6,701,102,b'Auckland',b'Wellington',[b'kidney', b'liver'],[])]), 25), 6, within(and([Functor(14504077,1,receive(b'Wellington', transplantOutcome(701, success))), Functor(14639885,6,701,102,b'Auckland',b'Wellington',[b'kidney', b'liver'],[])]), 9), dependent, :(b'Individual', :(b'Patients will not have to wait more than 25 time periods for a transplant. ID', 102))) ;
T = 22, Event = receive(b'Wellington', transplantOutcome(701, success)) .

At time period 29, Otago's expectation that patient 101 will not have to wait more than 25 time periods from the wait list acceptance message for an organ donation is violated.

In [12]:
?- happensAt("Otago", viol(_, _, _, ViolCondition, _, Message), 29).

ViolCondition = within(and([Functor(14504077,1,receive(_1700, transplantOutcome(_1706, _1708))), Functor(14639885,6,_1706,101,_1722,_1700,_1726,_1728)]), 0), Message = :(b'Individual', :(b'Patients will not have to wait more than 25 time periods for a transplant. ID', 101)) .

Note that the different agents were able to have different expectation rules which could be successfully trigger, violated or fulfilled by messages sent between the agents.

# Visualisation

We want to show that our prolog code can be used to produce visualisations. Note that the current code to enable visualisation is much longer than will be necessary once our kernel can be used to write output, rather than having to save query output.

Firstly, we declare some helper predicates. 

`allAgents/1` gives us all agents which have been declared at time zero, as well as **monSys**.
`maxTime/1` tells us which time period is the last one for which we have fluents holding or events occurring.
`visualisationLabel/3` contains reader friendly text strings describing different events / fluent and their arguments. The first argument is the event / fluent type, the second is a description of it, and the third argument is a list containing reader friendly names for each argument for the given event / fluent.

In [13]:
% File: helperVisualisation.pl
initialAgents(AgentList):- findall(Agent, initially(monSys:agent(Agent)), AgentList).
allAgents(AgentList):- initialAgents(InitialList), append(InitialList, [monSys], AgentList).

maxTime(Val):- findall(T, (holdsAt(_, _, T); happensAt(_,_,T)), TList), max_list(TList, Val).

visualisationLabel(waitAddReq, "Waiting list add request", ["Patient ID", "organs", "details"]).
visualisationLabel(waitAccept, "waiting list acceptance", ["Patient ID"]).
visualisationLabel(donorOffer, "Donation offer", ["Patient ID", "organs", "details"]).
visualisationLabel(acceptMatch, "Match acceptance", ["Match ID"]).
visualisationLabel(confirmedNotification, "Match confirmation", ["Match ID"]).
visualisationLabel(recipientFound, "Recipient found notification", ["Match ID", "recipient ID", "donor hospital", "transplant location", 
"organs", "details"]).
visualisationLabel(donorFound, "Donor found notification", ["Match ID", "donor ID", "recipient hospital", "transplant location", 
"organs", "details"]).
visualisationLabel(locationSelected, "Transplant location selected", ["Match ID", "donor hospital", "recipient hospital",
"organs", "details"]).
visualisationLabel(confirmationMatch, "Local match confirmation", ["Match ID"]).
visualisationLabel(transplant, "Transplant", ["Match ID", "outcome"]).
visualisationLabel(transplantOutcome, "Transplant outcome", ["Match ID", "outcome"]).
visualisationLabel(match, "Match", ["Match ID", "donor hospital", "donor ID", "recipient hospital", "recipient ID", "transplant location", "organs", "details"]).
visualisationLabel(waiting, "Waiting", ["Patient ID", "hospital", "organs", "details", "start time"]).

We save the results from these helper predicates into the **VisualisationSupport.txt** file. 

In [14]:
% Output: VisualisationSupport.txt
?- allAgents(Agents).
?- maxTime(MaxTime).
?- visualisationLabel(FluentName, FluentMessage, Arguments).

Agents = [ b'Otago', b'Christchurch', b'Wellington', b'Auckland', monSys ] .
MaxTime = 30 .
FluentName = waitAddReq, FluentMessage = b'Waiting list add request', Arguments = [ b'Patient ID', b'organs', b'details' ] ;
FluentName = waitAccept, FluentMessage = b'waiting list acceptance', Arguments = [ b'Patient ID' ] ;
FluentName = donorOffer, FluentMessage = b'Donation offer', Arguments = [ b'Patient ID', b'organs', b'details' ] ;
FluentName = acceptMatch, FluentMessage = b'Match acceptance', Arguments = [ b'Match ID' ] ;
FluentName = confirmedNotification, FluentMessage = b'Match confirmation', Arguments = [ b'Match ID' ] ;
FluentName = recipientFound, FluentMessage = b'Recipient found notification', Arguments = [ b'Match ID', b'recipient ID', b'donor hospital', b'transplant location', b'organs', b'details' ] ;
FluentName = donorFound, FluentMessage = b'Donor found notification', Arguments = [ b'Match ID', b'donor ID', b'recipient hospital', b'transplant location', b'organs', b'details'

We save the query output for events to **VisualisationEvents.txt**. Here, we use different argument names for the agents so that we can easily identify at the start of each line of output which type of event we are describing. While the output from this cell and other cells below is hidden in Deepnote for readibility, this formatting does not transfer to Github.

In [15]:
% Output: VisualisationEvents.txt

?- happensAt(SendAgent, Event, T), Event = send(_, _).
?- happensAt(ReceiveAgent, Event, T), Event = receive(_, _).
?- happensAt(ViolAgent, viol(_,_,_,_,_,Message), T).
?- happensAt(FulfAgent, fulf(_,_,_,_,_,Message), T).
?- happensAt(OtherAgent, Event, T), Event \= send(_, _), Event \= receive(_, _), Event \= viol(_, _, _, _, _, _), Event \= fulf(_, _, _, _, _, _).

SendAgent = b'Otago', Event = send(monSys, waitAddReq(101, [b'kidney', b'liver'], [Functor(188685,2,bloodType,b'O')])), T = 1 ;
SendAgent = b'Otago', Event = send(monSys, waitAddReq(102, [b'kidney', b'heart', b'liver'], [Functor(188685,2,bloodType,b'A')])), T = 3 ;
SendAgent = b'Wellington', Event = send(monSys, waitAddReq(103, [b'heart', b'pancreas'], [Functor(188685,2,bloodType,b'B')])), T = 5 ;
SendAgent = b'Wellington', Event = send(monSys, waitAddReq(104, [b'heart', b'pancreas', b'lungs'], [Functor(188685,2,bloodType,b'AB')])), T = 7 ;
SendAgent = b'Auckland', Event = send(monSys, donorOffer(201, [b'heart'], [Functor(188685,2,bloodType,b'A'), Functor(188685,2,condition,[b'HIV', b'Cancer'])])), T = 9 ;
SendAgent = b'Auckland', Event = send(monSys, donorOffer(202, [b'kidney', b'liver', b'pancreas'], [Functor(188685,2,bloodType,b'O')])), T = 11 ;
SendAgent = b'Auckland', Event = send(monSys, acceptMatch(701)), T = 15 ;
SendAgent = b'Wellington', Event = send(monSys, acceptMatch(701))

We also save the expectation rules that hold for our different agents. I have split the query into multiple time ranges to get around the current query length limit in the kernel. We would want to add the functionality to remove the query limit for certain cells so that it is guaranteed that all results are returned by a given query.

In [16]:
% Output: VisualisationRules.txt
?- holdsAt(ExpRuleAgent, exp_rule(_, _, _, Message), T), 0 =< T, T =< 9.
?- holdsAt(ExpRuleAgent, exp_rule(_, _, _, Message), T), 10 =< T, T =< 19.
?- holdsAt(ExpRuleAgent, exp_rule(_, _, _, Message), T), 20 =< T, T =< 29.

ExpRuleAgent = b'Otago', Message = :(b'Individual', :(b'Patients will not have to wait more than 25 time periods for a transplant. ID', _1740)), T = 0 ;
ExpRuleAgent = b'Wellington', Message = :(b'Individual', :(b'Patients will not be added to the waiting list for more than 2 organs at once. ID', _1742)), T = 0 ;
ExpRuleAgent = b'Auckland', Message = :(b'Individual', :(b'Donor offers will not be made for patients diagnosed with cancer. ID', _1742)), T = 0 ;
ExpRuleAgent = b'Otago', Message = :(b'Individual', :(b'Patients will not have to wait more than 25 time periods for a transplant. ID', _1740)), T = 1 ;
ExpRuleAgent = b'Wellington', Message = :(b'Individual', :(b'Patients will not be added to the waiting list for more than 2 organs at once. ID', _1742)), T = 1 ;
ExpRuleAgent = b'Auckland', Message = :(b'Individual', :(b'Donor offers will not be made for patients diagnosed with cancer. ID', _1742)), T = 1 ;
ExpRuleAgent = b'Otago', Message = :(b'Individual', :(b'Patients will not ha

We decide to also visualise `waiting/5` fluents, so must save the appropriate queries:

In [17]:
% Output: VisualisationWaiting.txt
?- holdsAt(FluentAgent, Fluent, T), Fluent = waiting(_,_,_,_,_), 0 =< T, T =< 9.
?- holdsAt(FluentAgent, Fluent, T), Fluent = waiting(_,_,_,_,_), 10 =< T, T =< 19.
?- holdsAt(FluentAgent, Fluent, T), Fluent = waiting(_,_,_,_,_), 20 =< T, T =< 29.

FluentAgent = monSys, Fluent = waiting(101, b'Otago', [b'kidney', b'liver'], [Functor(188685,2,bloodType,b'O')], 2), T = 3 ;
FluentAgent = monSys, Fluent = waiting(101, b'Otago', [b'kidney', b'liver'], [Functor(188685,2,bloodType,b'O')], 2), T = 4 ;
FluentAgent = monSys, Fluent = waiting(101, b'Otago', [b'kidney', b'liver'], [Functor(188685,2,bloodType,b'O')], 2), T = 5 ;
FluentAgent = monSys, Fluent = waiting(102, b'Otago', [b'kidney', b'heart', b'liver'], [Functor(188685,2,bloodType,b'A')], 4), T = 5 ;
FluentAgent = monSys, Fluent = waiting(101, b'Otago', [b'kidney', b'liver'], [Functor(188685,2,bloodType,b'O')], 2), T = 6 ;
FluentAgent = monSys, Fluent = waiting(102, b'Otago', [b'kidney', b'heart', b'liver'], [Functor(188685,2,bloodType,b'A')], 4), T = 6 ;
FluentAgent = monSys, Fluent = waiting(101, b'Otago', [b'kidney', b'liver'], [Functor(188685,2,bloodType,b'O')], 2), T = 7 ;
FluentAgent = monSys, Fluent = waiting(102, b'Otago', [b'kidney', b'heart', b'liver'], [Functor(188685,2,

We will use Graphviz in Python to create our visualisation example. 

Firstly, we save the information from **VisualisationSupport.txt** into a support dictionary, using regex to extract the useful values, and we define an array to store our written descriptions of events and fluents. Our array **arr** has 3 dimensions - one for the given agent, one for each type of event / fluent we are interested in displaying, and one for the time period.

In [18]:
% PYTHON
from re import compile, search
import numpy as np
from graphviz import Digraph

# Create support dictionary with labels for fluents, events
supportDict = {}
f1 = open("/work/output_files/VisualisationSupport.txt", "r")
lines = f1.readlines()
f1.close()
lineCount = 0
for line in lines:
    # Find the agents to visualise
    if lineCount == 0:
        agents = line.strip("Agents = [ ").strip(" ] .\n")
        agents = compile("b'(.+?)'").sub("\g<1>", agents).split(", ")
        print(agents)
        firstLine = False
    # Find maximum time period
    elif lineCount == 1:
        maxTime = int(compile("MaxTime = (\d+) .\n").sub("\g<1>", line))
    # Save supporting descriptions
    else:
        line = compile("FluentName = (.+), FluentMessage = (.+), Arguments = \[ (.+) ] [.|;]\n").sub("\g<1>;\g<2>;\g<3>", line)
        line = compile("b'(.+?)'[,|;][ ]*").sub("\g<1>;", line)
        line = compile("b'(.+?)'[ ]*").sub("\g<1>", line)
        supportDict[line[:line.index(";")]] = line[line.index(";")+1:]
    lineCount += 1
print(supportDict)

# Create a matrix to store all the written descriptions
arr = np.zeros((len(agents), 7,int(maxTime)+1), dtype=object)

['Otago', 'Christchurch', 'Wellington', 'Auckland', 'monSys']
{'waitAddReq': 'Waiting list add request;Patient ID;organs;details', 'waitAccept': 'waiting list acceptance;Patient ID', 'donorOffer': 'Donation offer;Patient ID;organs;details', 'acceptMatch': 'Match acceptance;Match ID', 'confirmedNotification': 'Match confirmation;Match ID', 'recipientFound': 'Recipient found notification;Match ID;recipient ID;donor hospital;transplant location;organs;details', 'donorFound': 'Donor found notification;Match ID;donor ID;recipient hospital;transplant location;organs;details', 'locationSelected': 'Transplant location selected;Match ID;donor hospital;recipient hospital;organs;details', 'confirmationMatch': 'Local match confirmation;Match ID', 'transplant': 'Transplant;Match ID;outcome', 'transplantOutcome': 'Transplant outcome;Match ID;outcome', 'match': 'Match;Match ID;donor hospital;donor ID;recipient hospital;recipient ID;transplant location;organs;details', 'waiting': 'Waiting;Patient ID;h

The `extractArguments` method creates a list of arguments from some text output will still keeping bracketed information together.

In [19]:
% PYTHON
# Make list of arguments while still keeping arguments in brackets together
def extractArguments(contents):
    squareBracket = 0
    circleBracket = 0
    value = ""
    values = []
    for char in contents:
        if char == '[':
            squareBracket += 1
        elif char == ']':
            squareBracket -= 1
        elif char == "(":
            circleBracket += 1
        elif char == ")":
            circleBracket -= 1
        elif char == ",":
            if circleBracket == 0 and squareBracket == 0:
                values.append(value.strip(" "))
                value = ""
        else:
            value += char
    if contents[-1] != ",":
        values.append(value.strip(" "))
    return values


The `findLabel` method takes a previously saved message and formats it to fit in our agent boxes.

In [20]:
% PYTHON


def findLabel(arr, agent, agents, typeIndex, time):
    counter = 0
    message = "{"
    if arr[agents.index(agent), typeIndex, time] == 0:
        return ""
    for entry in arr[agents.index(agent), typeIndex, time]:
        charCounter = 0
        entryFormatted = ""
        if agent == "monSys":
            maxChar = 70
        else:
            maxChar = 40
        for char in entry:
            entryFormatted += char

            if charCounter > maxChar and char == " ":
                entryFormatted += "\\l"
                charCounter = 0
            charCounter += 1
        entryFormatted += "\\l"
        message += "<f" + str(counter) + ">" + entryFormatted + "| "
        counter += 1
    message = message[:-2]
    message += "}"
    return message

Here, we save the saved output from events and fluents into our array, using regex to extract useful values, and then alternating variable arguments with their written descriptor in our saved messages.

In [21]:
% PYTHON

# Read in output files with fluents, events of interest
f = open("/work/output_files/VisualisationEvents.txt", "r")
lines = f.readlines()
f.close()
f = open("/work/output_files/VisualisationRules.txt", "r")
lines += f.readlines()
f.close()
f = open("/work/output_files/VisualisationWaiting.txt", "r")
lines += f.readlines()
f.close()
# Defined types
eventTypes = ["send", "receive", "violation", "fulfilment", "other", "expectation rule", "fluent"]
for line in lines:
    # Simplify Deepnote formatting
    line = compile('Functor\([\d]+,2,(.+?),(.+?)\)').sub('\g<1>:\g<2>', line)
    line = compile("b'(.+?)'").sub("\g<1>", line)
    firstArg = line.split(" ")[0]
    # Choose index for matrix
    if firstArg == "SendAgent":
        eIndex = 0
    elif firstArg == "ReceiveAgent":
        eIndex = 1
    elif firstArg == "ViolAgent":
        eIndex = 2
    elif firstArg == "FulfAgent":
        eIndex = 3
    elif firstArg == "ExpRuleAgent":
        eIndex = 5
    elif firstArg == "FluentAgent":
        eIndex = 6
    else:
        eIndex = 4
    # If event or fluent is not a specific type
    if eventTypes[eIndex] == "other" or eventTypes[eIndex] == "fluent":
        # Extract arguments of interest
        if eventTypes[eIndex] == "fluent":
            line = compile('.+?Agent = (.+?), Fluent = (.+?)\((.+?)\), T = (.+?) [.|;]\n').sub('\g<1>;\g<2>;\g<3>;\g<4>', line)
        else:
            line = compile('.+?Agent = (.+?), Event = (.+?)\((.+?)\), T = (.+?) [.|;]\n').sub('\g<1>;\g<2>;\g<3>;\g<4>', line)
        
        lineSplit = line.split(";")
        # Find current agent, time and message type
        currentAgent = lineSplit[0]
        currentTime = int(lineSplit[-1])
        messageType = lineSplit[1]
        # Process main fluent / event
        contents = lineSplit[2]
        values = extractArguments(contents)
        argNames = supportDict[messageType].split(";")
        message = argNames[0] + ". "

    if eventTypes[eIndex] == "send" or eventTypes[eIndex] == "receive":
         # Extract arguments of interest
        line = compile('.+?Agent = (.+?), Event = .+?\((.+?), (.+?)\), T = (.+?) [.|;]\n').sub('\g<1>;\g<2>;\g<3>;\g<4>', line)
        message = ""
        if (eventTypes[eIndex] == "send"):
            message = "To: " 
        elif (eventTypes[eIndex] == "receive"):
            message = "From: "
        lineSplit = line.split(";")
        # Find current agent, time and message type
        currentAgent = lineSplit[0]
        currentTime = int(lineSplit[3])
        contents = lineSplit[2]
        # Process main fluent / event
        messageType = contents[:contents.index('(')]
        contents = contents[contents.index('(')+1:-1]
        values = extractArguments(contents)
        argNames = supportDict[messageType].split(";")
        message += lineSplit[1] + ", " + argNames[0] + ". "

    # Apply appropriate label to each argument
    if eventTypes[eIndex] == "send" or eventTypes[eIndex] == "receive" or eventTypes[eIndex] == "other" or eventTypes[eIndex] == "fluent":
        for pos in range(1, len(argNames)):
            message += argNames[pos] + ": " + values[pos-1]+ ", "
        message = message[:-2]
        # Save in matrix
        if arr[agents.index(currentAgent), eIndex, currentTime] == 0:
            arr[agents.index(currentAgent), eIndex, currentTime] = [message]
        else:
            arr[agents.index(currentAgent), eIndex, currentTime].append(message)
            
    # If most of output is the rule message, save that for display        
    elif eventTypes[eIndex] == "fulfilment" or eventTypes[eIndex] == "violation" or eventTypes[eIndex] == "expectation rule":
        currentTime = int(line.split()[-2])
        message = ""
        if eventTypes[eIndex] == "expectation rule":
            line = compile(".+?Agent = (.+?), Message = :\((.+?), :\((.+) .+?, .+\)\), T = (.+?) [.|;]\n").sub('\g<1> \g<2> - \g<3>', line)
        else:
            line = compile(".+?Agent = (.+?), Message = :\((.+?), :\((.+?), (.+)\)\), T = .+?[;|.]\n").sub('\g<1> \g<2> - \g<3> \g<4>', line)
        currentAgent = line.split()[0]
        message = line[line.index(" ")+1:]
        if arr[agents.index(currentAgent), eIndex, currentTime] == 0:
            arr[agents.index(currentAgent), eIndex, currentTime] = [message]
        else:
            arr[agents.index(currentAgent), eIndex, currentTime].append(message)

We create a PNG file for each time period from zero up to our maximum time period. Each graph contains a cluster for each agent, and within a given agent, nodes are shown for events and fluents. Getting the layout of subclusters in graphviz can be somewhat challenging. Many further design tweaks are possible, and changes should be made, but this code gives us an idea of what can be produced. The PNGs are saved in the **Images** subfolder.

In [22]:
% PYTHON

for currentTime in range(maxTime):
    g = Digraph(node_attr={'shape':'box', 'style': 'rounded,filled'})
    g.graph_attr["label"] = "Example scenario. Time: "+str(currentTime)
    g.node_attr["shape"] = "box"
    g.node_attr["fontsize"] = '13'
    g.node_attr["style"] = "filled"
    g.format = 'png'
    # We set the connections between clusters to be invisible so that they determine the layout without being visually distracting
    g.edge_attr["style"] = "invis"
    pos = 0
    previousNode1 = ""
    previousNode0 = ""
    # NOTE: the subgraph name needs to begin with 'cluster' (all lowercase)
    #       so that Graphviz recognizes it as a special cluster subgraph
    for agent in agents:
        clusterName = "cluster_" + agent
        with g.subgraph(name=clusterName) as agentGraph:
            # Appearance
            agentGraph.attr(shape='box', style= 'rounded,filled', color='#E8ECFB', label=agent, rankdir="LR")
            agentGraph.node_attr["fixedsize"] = 'true'
            if agent == "monSys":
                agentGraph.node_attr["height"] = '3'
                agentGraph.node_attr["width"] = '6'
            else:
                agentGraph.node_attr["height"] = '2'
                agentGraph.node_attr["width"] = '4'
            agentGraph.node_attr["style"] = 'solid, filled'
            agentGraph.node_attr["color"] = "black"
            agentGraph.node_attr["fillcolor"] = "#EEEEEE"

            # Subgraphs for each agent. Fulfilments, Violations, Rules, Other Events, Sent, Received.
            with agentGraph.subgraph(name='cluster_Fulf') as fulf:
                nodeName = agent + "Fulf"
                nodeLabel = findLabel(arr, agent, agents, 3, currentTime)
                fulf.attr(shape='box', style= 'rounded, filled', color='#CCDDAA', penwidth = '3', label='Fulfilments')
                fulf.node(nodeName, label=nodeLabel, shape="record")
            
            with agentGraph.subgraph(name='cluster_Viol') as viol:
                nodeName = agent + "Viol"
                nodeLabel = findLabel(arr, agent, agents, 2, currentTime)
                viol.attr(shape='box', style= 'rounded, filled', color='#FFCCCC', penwidth = '3', label='Violations')
                viol.node(nodeName, label=nodeLabel, shape="record")

            with agentGraph.subgraph(name='cluster_Rules') as rules:
                nodeName = agent + "Rules"
                nodeLabel = findLabel(arr, agent, agents, 5, currentTime)
                rules.attr(shape='box', style= 'rounded, filled', color='#D9CCE3', penwidth = '3', label='Expectation rules')
                rules.node(nodeName, label=nodeLabel, shape="record")


            with agentGraph.subgraph(name='cluster_Other') as another:
                nodeLabel = findLabel(arr, agent, agents, 4, currentTime)
                nodeName = agent + "Other"
                another.attr(shape='box', style= 'rounded, filled', color='#D9CCE3', penwidth = '3', label='Other Events')
                another.node(nodeName, label=nodeLabel, shape="record")

            with agentGraph.subgraph(name='cluster_Sent') as sent:
                nodeName = agent + "Sent"
                nodeLabel = findLabel(arr, agent, agents, 0, currentTime)
                sent.attr(shape='box', style= 'rounded, filled', color='#D9CCE3', penwidth = '3', label='Sent')
                sent.node(nodeName, label=nodeLabel, shape="record")

            with agentGraph.subgraph(name='cluster_Received') as received:
                nodeName = agent + "Received"
                nodeLabel = findLabel(arr, agent, agents, 1, currentTime)
                received.attr(shape='box', style= 'rounded, filled', color='#D9CCE3', penwidth = '3', label='Received')
                received.node(nodeName, label=nodeLabel, shape="record")
            
            # Layout within each agent subgraph
            agentGraph.edge_attr["style"] = "invis"
            agentGraph.edge(agent + "Received",agent + "Sent")
            agentGraph.edge(agent + "Fulf",agent + "Viol")
            agentGraph.edge(agent + "Other",agent + "Rules")

            # Show waiting fluents for monSys
            if agent == "monSys":
                with agentGraph.subgraph(name='cluster_Wait') as wait:
                    nodeName = agent + "Wait"
                    nodeLabel = findLabel(arr, agent, agents, 6, currentTime)
                    wait.attr(shape='box', style= 'rounded, filled', color='#D9CCE3', penwidth = '3', label='Wait List')
                    wait.node(nodeName, label=nodeLabel, shape="record")

        # Control layout of agents
            innerNode = agent + "Sent"
        if agent != "monSys":
            if (pos % 2 == 0 and previousNode0 != ""): 
                g.edge(innerNode,previousNode0)
                previousNode0 = agent + "Received"
            elif (pos % 2 == 1 and previousNode1 != ""):
                g.edge(innerNode,previousNode1)
            if pos % 2 == 0:
                previousNode0 = agent + "Received"
            else:
                previousNode1 = agent + "Received"
        pos = pos + 1
    # Add monSys to the top 
    g.edge("monSysSent",previousNode1)
    g.edge("monSysViol",previousNode0)
    # Save to Images subfolder
    if currentTime < 10:
        fname = "/work/Visualisation/Images/time-0" + str(currentTime)
    else:
        fname = "/work/Visualisation/Images/time-" + str(currentTime)
    g.render(filename=fname)

While each PNG shows the view for each of the agents, you can imagine that a given agent would only be able to view their own subgraph. It is possible to concatenate the PNGs into a GIF, or to use them in an image slider (not currently supported in Deepnote). All of the PNGs are available in the Images folder, with the PNG files for time periods 5, 7 and 11 shown below. The expectation rule message labels are used for fulfilments, violations and expectation rules, while the sent, received and wait list boxes make use of the labels defined in the `visualisationLabel/3` fluent.

<img src="https://github.com/katetruman/MultiAgentEC/blob/master/Visualisation/Images/time-05.png?raw=true" width="2000">

------
<img src="https://github.com/katetruman/MultiAgentEC/blob/master/Visualisation/Images/time-07.png?raw=true" width="2000">


-----
<img src="https://github.com/katetruman/MultiAgentEC/blob/master/Visualisation/Images/time-11.png?raw=true" width="2000">

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=32f94018-a4da-40ef-8c9f-8983d73811c8' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>